In [7]:
# %load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: daniel_v1.6.5.

In [8]:
import enum
import json
import time
import threading

import traitlets
import ipywidgets as ipw
from ipywidgets import widgets, Layout

from aiidalab_widgets_base import (
    ComputationalResourcesWidget,
    ProcessMonitor,
    ProcessNodesTreeWidget,
    WizardAppWidget,
    WizardAppWidgetStep,
)

from aiida.orm import Code, ArrayData
from aiida.engine import run_get_node, submit, ProcessState
from aiida.common import NotExistent
from aiidalab_widgets_base import viewer
from IPython.display import clear_output


# Taken from aiidalab_qe (this feels like a method that should be moved to aiidalab_widgets_base) 
class NodeViewWidget(ipw.VBox):

    node = traitlets.Instance(Node, allow_none=True)

    def __init__(self, **kwargs):
        self._output = ipw.Output()
        super().__init__(children=[self._output], **kwargs)

    @traitlets.observe("node")
    def _observe_node(self, change):
        if change["new"] != change["old"]:
            with self._output:
                clear_output()
                if change["new"]:
                    display(viewer(change["new"]))

In [9]:
##############################################################################
#The app works around 4 major steps
#    (1) Configure the user input
#    (2) Confirm the user input + submit the job
#    (3) Monitor the process during the run
#    (4) Display the final results
#
#These steps are linked via the following 
#    (1) --> (2): user_inputs, a dictionary of all the user inputs
#    (1) --> (2): mpuc3_code, a code for marketplace usercase 3
#    (2) --> (3): process, the process (MPusercase3 CalcJob) submitted to AiiDA 
#    (3) --> (4): output, the ArrayData output from the CalcJob
#
#This is an early prototype that is 'stapled' from the pizza example in aiidalab_widgets_base
#as well as QE example from aiidalab_qe
#
#Specific notes/issues on the steps:
#    Step 1:
#        * Perhaps there should be a step 0 that allows the user to choose a model,
#          then the widget can be dynamically updated
#        * The code setup is rather ugly, and creating a custom code crashes
#        * The user should be alerted that if they try to hit submit without 
#          choosing a code, that they need to choose a code first
#    Step 2:
#    Step 3:
#        * This is taken almost directly from QE, and is the part I understand the least
#        * Clicking on most outputs causes a crash
#        *** I can't seem to set the 'output' propery correctly
#    Step 4:
#        *** Because I can't set the output property in step3 correctly, this step displays nothing
#        * The current display is very ugly and taken directly from Step2, and probably be changed to
#         something nicer
##############################################################################

In [17]:
import ipywidgets as widgets
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            # 'width': '100%',
            # 'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

handler.show_logs()

handler.clear_logs()
logger.info('Starting program')

# try:
#     logger.info('About to try something dangerous...')
#     1.0/0.0
# except Exception as e:
#     logger.exception('An error occurred!')

Output(layout=Layout(border='1px solid black'))

In [27]:
## logging wrapper for debugging
out = widgets.Output(layout={'border': '1px solid black'})
#@out.capture() #add this decorator to functions that you want to log
handler.clear_logs()

handler.show_logs()

handler.clear_logs()
logger.info('Starting program')

class ConfigureUserInputStep(ipw.VBox, WizardAppWidgetStep):

    disabled = traitlets.Bool()
    user_inputs = traitlets.Dict(allow_none=True)
    mpuc3_code = traitlets.Instance(Code, allow_none=True)

    def __init__(self, **kwargs):

        # create code selection field
        self.code_selector = ComputationalResourcesWidget(
            description="code", input_plugin="marketusercase3"
        )
        self.code_selector.observe(self._set_code_value, ["value"])
        self.code_selector.observe(self._update_state, ["value"])

        # create fields to enter user values
        self.description_label_default =[
        ["ATSBcons", "ATSB Concentration", 1.94],
        ["Precurfr", "Precursor Volume Flow Rate", 40.0],
        ["Dispfr", "Dispersion Volume Flow Rate", 72.0],
        ["Pilotch4fr", "Pilot Methane Volume Flow Rate", 4.0],
        ["Piloto2fr", "Pilot Oxygen Volume Flow Rate", 8.0],
        ["Fanrate", "Fan Extraction Volume Flow Rate", 270.0],
        ]
        def _setup_input_entry(description,init_value):
                form_item_layout = Layout(justify_content='space-between')
                form = widgets.HBox([widgets.Label(value=description),
                             widgets.FloatText(value=init_value)],
                             layout=form_item_layout)
                return form
        items_layout = Layout(padding='20px', align_items='stretch', width='75%')
        self.input_widgets = [_setup_input_entry(x[1],x[2]) for x in self.description_label_default]

        # create confirmation button
        self.confirm_button = ipw.Button(
            description="Confirm user_inputs",
        )
        self.confirm_button.on_click(self._confirm_user_inputs)
        self.observe(self._update_state, ["user_inputs"])

        # setup widget
        super().__init__([self.code_selector] + self.input_widgets+[self.confirm_button],
                         layout=items_layout, **kwargs)

    def _set_code_value(self,_):
        print("set_code_value, code: ",self.mpuc3_code)
        self.mpuc3_code = self.code_selector.value
        print("set_code_value (after), code: ",self.mpuc3_code)

    def _confirm_user_inputs(self, button):
        label_remapdict = {x[1]:x[0] for x in self.description_label_default}
        user_inputs = {label_remapdict[x.children[0].value]: x.children[1].value for x in self.input_widgets}

        self.user_inputs = user_inputs 
        
    def reset(self):
        with self.hold_trait_notifications():
            self.style.value = None
            self.toppings.value = []
            self.user_inputs = {}
            
    @traitlets.default("state")
    def _default_state(self):
        return self.State.READY

    def _update_state(self, _=None):
        """Update the step's state based on traits and widget state.

        The step state influences the representation of the step (e.g. the "icon") and
        whether the "Next step" button is enabled.
        """

        if self.user_inputs and self.mpuc3_code:
            # The configuration is non-empty, we can move on to the next step.
            self.state = self.State.SUCCESS
        else:
            # In all other cases the step is always considered to be in the "init" state.
            self.state = self.State.READY

    @traitlets.observe("state")
    def _observe_state(self, change):
        with self.hold_trait_notifications():
            self.disabled = change["new"] == self.State.SUCCESS
            self.confirm_button.disabled = change["new"] is not self.State.CONFIGURED
        
    @traitlets.observe("disabled")
    def _observe_disabled(self, change):
        with self.hold_trait_notifications():
            for child in self.children:
                child.disabled = change["new"]




class ConfirmUserInputStep(ipw.VBox, WizardAppWidgetStep):

    process = traitlets.Instance(ProcessNode, allow_none=True)
    submit = traitlets.Bool()
    user_inputs = traitlets.Dict(allow_none=True)
    mpuc3_code = traitlets.Instance(Code, allow_none=True)

    def __init__(self, **kwargs):
        self.userinputs_label = ipw.HTML()

        # The second step has only function: executing the order by clicking on this button.
        self.submitcalc_button = ipw.Button(description="Submit calculation")
        self.submitcalc_button.on_click(self.submit_calc)
        super().__init__([self.userinputs_label, self.submitcalc_button], **kwargs)
        
    def reset(self):
        self.submit = None

    @traitlets.observe("user_inputs")
    def _observe_configuration(self, change):
        "Format and show the user inputs."
        if change["new"]:
            self.userinputs_label.value = f"<h4>Configuration</h4><pre>{json.dumps(change['new'], indent=2)}</pre>"
        else:
            self.userinputs_label.value = (
                "<h4>Configuration</h4>[Please configure user inputs]"
            )
    
    def submit_calc(self, button):
        fluent_calcjob = CalculationFactory('marketusercase3')
        fluentcalc_builder = fluent_calcjob.get_builder()
        fluentcalc_builder.user_inputs = Dict(dict=self.user_inputs)
        fluentcalc_builder.code = self.mpuc3_code 
        #fluentcalc_builder.metadata.dry_run = True
        self.process = submit(fluentcalc_builder)
        self.submit = True
        self._update_state()
        return

    def _update_state(self, _=None):
        "Update the step's state based on the order status and configuration traits."
        if self.submit:  # the order has been submitted
            self.state = self.State.SUCCESS
        elif self.user_inputs:  # the order can be submitted
            self.state = self.State.CONFIGURED
        else:
            self.state = self.State.INIT

    @traitlets.observe("state")
    def _observe_state(self, change):
        """Enable the order button once the step is in the "configured" state."""
        #self.submitcalc_button.disabled = change["new"] != self.State.CONFIGURED

class MonitorProcessStep(ipw.VBox, WizardAppWidgetStep):

    process = traitlets.Instance(ProcessNode, allow_none=True)
    output = traitlets.Instance(ArrayData, allow_none=True)

    def __init__(self, **kwargs):
        self.process_tree = ProcessNodesTreeWidget()
        ipw.dlink((self, "process"), (self.process_tree, "process"))

        self.node_view = NodeViewWidget(layout={"width": "auto", "height": "auto"})
        ipw.dlink(
            (self.process_tree, "selected_nodes"),
            (self.node_view, "node"),
            transform=lambda nodes: nodes[0] if nodes else None,
        )
        self.process_status = ipw.VBox(children=[self.process_tree, self.node_view])

        # Setup process monitor
        self.process_monitor = ProcessMonitor(
            timeout=0.2,
            callbacks=[
                self.process_tree.update,
                self._update_state,
            ],
        )
        ipw.dlink((self, "process"), (self.process_monitor, "process"))

        super().__init__([self.process_status], **kwargs)

    def can_reset(self):
        "Do not allow reset while process is running."
        return self.state is not self.State.ACTIVE

    def reset(self):
        self.process = None


    def _update_state(self):
        logger.info('M: start')
        if self.process is None:
            self.state = self.State.INIT
        else:
            process_state = self.process.process_state
            if process_state in (
                ProcessState.CREATED,
                ProcessState.RUNNING,
                ProcessState.WAITING,
            ):
                logger.info('M: ACTIVE')
                self.state = self.State.ACTIVE
            elif process_state in (ProcessState.EXCEPTED, ProcessState.KILLED):
                logger.info('M: KILLED')
                self.state = self.State.FAIL
            elif process_state is ProcessState.FINISHED:
                logger.info('M: SUCCESS 1')
                self.state = self.State.SUCCESS
                logger.info('M: SUCCESS 2')
                logger.info(self.process)
                try:
                    self.output = self.process.outputs.output
                except Exception as e:
                    logger.exception(e)


    @traitlets.observe("process")
    def _observe_process(self, change):
        print("MONITOR STATE", self.state)
        self._update_state()

class DisplayFinalOutput(ipw.VBox, WizardAppWidgetStep):

    output = traitlets.Instance(ArrayData, allow_none=True)

    def __init__(self, **kwargs):
        self.final_output = ipw.HTML()

        super().__init__([self.final_output], **kwargs)

    @traitlets.observe("output")
    def _call_observe(self,change):
        self._observe_configuration(self, change)

    def _observe_configuration(self, _, change):
        "Format and show the user inputs."
        if change["new"]:
            output = self.output
            area_flux = float(output.get_array('area_flux'))
            volume_flux = float(output.get_array('volume_flux'))
            particle_size = float(output.get_array('particle_size'))
            display_dict = {
                'Area Flux': area_flux,
                'Volume Flux': volume_flux,
                'Particle Size': particle_size,
            }
            self.final_output.value = f"<h4>Configuration</h4><pre>{json.dumps(display_dict, indent=2)}</pre>"
        else:
            self.final_output.value = (
                "<h4>Configuration</h4>[Please configure user inputs]"
            )



configure_userinput_step = ConfigureUserInputStep(auto_advance=True)
confirm_userinput_step = ConfirmUserInputStep(auto_advance=True)
monitorprocess_step = MonitorProcessStep(auto_advance=True)
displayfinaloutput_step = DisplayFinalOutput(auto_advance=True)

ipw.dlink(
    (configure_userinput_step, "user_inputs"),
    (confirm_userinput_step, "user_inputs"),
)
ipw.dlink(
    (configure_userinput_step, "mpuc3_code"),
    (confirm_userinput_step, "mpuc3_code"),
)
ipw.dlink(
    (confirm_userinput_step, "process"),
    (monitorprocess_step, "process"),
)
ipw.dlink(
    (monitorprocess_step, "output"),
    (displayfinaloutput_step, "output"),
)

# Setup the app by adding the various steps in order.
app = WizardAppWidget(
    steps=[
        ("Configure User Input", configure_userinput_step),
        ("Confirm User Input", confirm_userinput_step),
        ("Monitor Process", monitorprocess_step),
        ("Display Results", displayfinaloutput_step),
    ]
)

# Display the app to the user.
#from Ipython.display import display
# display(out)
display(app)

Output(layout=Layout(border='1px solid black'))

WizardAppWidget(children=(HBox(children=(Button(description='Previous step', disabled=True, icon='step-backwar…